# Import libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd
import re
import numpy as np
import time
import jupytext

# Define links

In [2]:
Countries_1stLeagueName = ['premier-league', 'bundesliga', 'serie-a', 'ligue-1', 'laliga']

In [3]:
table_urls = ['https://www.transfermarkt.com/premier-league/tabelle/wettbewerb/GB1?saison_id=2021', 
              'https://www.transfermarkt.com/bundesliga/tabelle/wettbewerb/GR1?saison_id=2021',
              'https://www.transfermarkt.com/serie-a/tabelle/wettbewerb/IT1?saison_id=2021',
              'https://www.transfermarkt.com/ligue-1/tabelle/wettbewerb/FR1?saison_id=2021',
              'https://www.transfermarkt.com/laliga/tabelle/wettbewerb/ES1?saison_id=2021']

# Request and parse league pages

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36',
           "Accept-Language": "en-US,en;q=0.5"}

In [12]:
responses = []
for url in table_urls:
    responses.append(requests.get(url=url, headers=headers))
    time.sleep(random.random())

In [14]:
soups = [BeautifulSoup(page.content, 'html.parser') for page in responses]

# Extract details from parsed pages

In [106]:
results = []
for soup in soups:
    table = soup.select('div.responsive-table')[0]
    teams = table.select('td.no-border-links.hauptlink')
    titles = [team.find('a').get('title') for team in teams]
    ids = [team.find('a').get('href').split('verein/')[1].split('/')[0] for team in teams]
    rows = table.select('tr')[1:]
    game_counts = [row.select('td.zentriert')[1].get_text() for row in rows]
    points = [row.select('td.zentriert')[7].get_text() for row in rows]

    for i in range(len(teams)):
        results.append({'team_title':titles[i], 'team_id':int(ids[i]), 'game_count':int(game_counts[i]), 'points':int(points[i])})


In [107]:
df = pd.DataFrame(results)

# Request UEFA champoions page in order to get UEFA members

In [108]:
uefa_paricipitants_url = 'https://www.transfermarkt.com/uefa-champions-league/teilnehmer/pokalwettbewerb/CL/saison_id/2021'

In [84]:
uefa_response = requests.get(url=uefa_paricipitants_url, headers=headers)

In [85]:
uefa_soup = BeautifulSoup(uefa_response.content, 'html.parser')

In [87]:
uefa_tags = uefa_soup.select('td.links.no-border-links.hauptlink a')

In [88]:
len(uefa_tags)

32

In [118]:
uefa_titles = [tag.get('title') for tag in uefa_tags]
uefa_ids = [int(tag.get('href').split('verein/')[1].split('/')[0]) for tag in uefa_tags]

In [109]:
df

,team_title,team_id,game_count,points
0,Manchester City,281,38,93
1,Liverpool FC,31,38,92
2,Chelsea FC,631,38,74
3,Tottenham Hotspur,148,38,71
4,Arsenal FC,11,38,69
...,...,...,...,...
89,RCD Mallorca,237,38,39
90,Cádiz CF,2687,38,39
91,Granada CF,16795,38,38
92,Levante UD,3368,38,35


# Add UEFA memberation status and define performance metric

In [122]:
df['uefa_status'] = df.team_id.isin(uefa_ids)
df['team_performance'] = df.points/df.game_count

In [123]:
df

,team_title,team_id,game_count,points,uefa_status,team_performance
0,Manchester City,281,38,93,True,2.447368
1,Liverpool FC,31,38,92,True,2.421053
2,Chelsea FC,631,38,74,True,1.947368
3,Tottenham Hotspur,148,38,71,False,1.868421
4,Arsenal FC,11,38,69,False,1.815789
...,...,...,...,...,...,...
89,RCD Mallorca,237,38,39,False,1.026316
90,Cádiz CF,2687,38,39,False,1.026316
91,Granada CF,16795,38,38,False,1.000000
92,Levante UD,3368,38,35,False,0.921053


In [4]:
df.to_csv('tables.csv', index=False)

# Process of finding claensheet for each club

In [6]:
df

,team_title,team_id,game_count,points,uefa_status,team_performance
0,Manchester City,281,38,93,True,2.447368
1,Liverpool FC,31,38,92,True,2.421053
2,Chelsea FC,631,38,74,True,1.947368
3,Tottenham Hotspur,148,38,71,False,1.868421
4,Arsenal FC,11,38,69,False,1.815789
...,...,...,...,...,...,...
89,RCD Mallorca,237,38,39,False,1.026316
90,Cádiz CF,2687,38,39,False,1.026316
91,Granada CF,16795,38,38,False,1.000000
92,Levante UD,3368,38,35,False,0.921053


## Import data of matches for season 20/21

In [7]:
competitons_df = pd.read_csv('competions.csv')

In [8]:
competitons_df

,host_id,host_status,score,match_id,match_date,guest_id,Season,goals_for,goals_against
0,583,Win,2:1,3589530,2022-04-17,244,2021,2,1
1,583,Win,2:0,3589292,2021-12-12,162,2021,2,0
2,583,Win,1:0,3589388,2022-02-11,273,2021,1,0
3,583,equal,0:0,3589262,2021-12-01,417,2021,0,0
4,583,Win,4:2,3588789,2021-08-14,667,2021,4,2
...,...,...,...,...,...,...,...,...,...
1442,65,Lost,0:1,3588246,2021-10-16,80,2021,0,1
1443,65,equal,0:0,3588390,2021-12-18,167,2021,0,0
1444,65,equal,0:0,3588408,2022-01-08,79,2021,0,0
1445,65,Win,2:1,3588471,2022-02-12,44,2021,2,1


In [10]:
id_df

,team_title,team_id
0,Manchester City,281
1,Liverpool FC,31
2,Chelsea FC,631
3,Tottenham Hotspur,148
4,Arsenal FC,11
...,...,...
89,RCD Mallorca,237
90,Cádiz CF,2687
91,Granada CF,16795
92,Levante UD,3368


## Merging team_title to competition

In [11]:
merged1 = pd.merge(competitons_df, id_df, left_on = "host_id", right_on = "team_id")
merged1.rename(columns={'team_title':'host_title'}, inplace =True)
merged1.drop(columns=['team_id'], inplace=True)

In [12]:
# team_id that they aren't in seasion 20/21
competitons_df[~(competitons_df.host_id.isin(id_df.team_id))].host_id.unique()

array([   27,    16,    15, 23826,    89,    60,     3,    39,   533,
          18,    24,    82,    80,   167,    79,    44,    10,    65],
      dtype=int64)

In [13]:
merged2 = pd.merge(merged1, id_df, left_on = "guest_id", right_on = "team_id")
merged2.rename(columns={'team_title':'guest_title'}, inplace =True)
merged2.drop(columns=['team_id'], inplace=True)

## Split goals for each home and guest team

In [15]:
merged2['home_score'] = [int(element[0]) for element in merged2.score.str.split(':')]
merged2['guest_score'] = [int(element[1]) for element in merged2.score.str.split(':')]

##  Make records for each home and guest team

In [18]:
teams1 = merged2[['match_id', 'host_id', 'host_title', 'home_score']].copy()
teams1.rename(columns={'host_id':'team_id', 'host_title':'team_title', 'home_score':'score'}, inplace=True)
teams1['team_status'] = 'home'

In [19]:
teams2 = merged2[['match_id', 'guest_id', 'guest_title', 'guest_score']].copy()
teams2.rename(columns={'guest_id':'team_id', 'guest_title':'team_title', 'guest_score':'score'}, inplace=True)
teams2['team_status'] = 'guest'

In [20]:
edited_competitions = pd.concat([teams1, teams2], axis=0).sort_values(by='match_id').reset_index(drop=True)

In [22]:
edited_competitions

,match_id,team_id,team_title,score,team_status
0,3583476,11,Arsenal FC,0,guest
1,3583476,1148,Brentford FC,2,home
2,3583477,1237,Brighton & Hove Albion,2,guest
3,3583477,1132,Burnley FC,1,home
4,3583478,631,Chelsea FC,3,home
...,...,...,...,...,...
2275,3718646,430,ACF Fiorentina,1,guest
2276,3719028,6195,SSC Napoli,0,home
2277,3719028,3522,Spezia Calcio,1,guest
2278,3751557,380,US Salernitana 1919,1,guest


## Sum cleansheets for each team

In [23]:
matches_wtih_0score = edited_competitions[edited_competitions.score==0].match_id

In [24]:
has_cleansheet = edited_competitions[edited_competitions.match_id.isin(matches_wtih_0score) & edited_competitions.score!=0]

In [25]:
team_claensheet = has_cleansheet.groupby(['team_id'])[['match_id']].apply('count').sort_values(by='match_id', ascending=False)

In [26]:
len(team_claensheet)

59

In [27]:
team_claensheet = pd.merge(team_claensheet, id_df, left_on='team_id', right_on='team_id')

In [28]:
len(team_claensheet)

59

## Save results as CSV

In [29]:
team_claensheet.rename(columns={'match_id':'cleansheets'}, inplace=True)

In [30]:
team_claensheet

,team_id,cleansheets,team_title
0,281,12,Manchester City
1,631,11,Chelsea FC
2,31,11,Liverpool FC
3,148,10,Tottenham Hotspur
4,5,10,AC Milan
5,667,9,RC Strasbourg Alsace
6,6195,8,SSC Napoli
7,12,8,AS Roma
8,398,7,SS Lazio
9,11,7,Arsenal FC


In [32]:
team_claensheet.to_csv('team_claensheet_seasion(21-22).csv', index =False)